In [1]:
import pandas as pd

df = pd.read_csv("data/data.csv")
df.head()

,SYM Im adding more every month up till the point this hits 50 SYM after that I ll start working on my SYM SYM and SYM,Bullish
0,SYM SYM SYM SYM SYM FEDs balance sheet total a...,Bearish
1,SYM SYM SYM SYM SYM X,Bullish
2,SYM going down to 300 in next 2 weeks,Bearish
3,How will this affect the market SYM price may ...,Bullish
4,SYM pos,Bullish


In [2]:
len(df)

37531

In [3]:
# -*- coding: utf-8 -*-
import requests
import json
from json import JSONDecodeError
import re
import datetime
from datetime import date, timedelta
import time
import sys
import threading
import random

def printToFile(linesOfData):
    with open('data/data.csv','a') as file:
        for line in linesOfData:
            file.write(line)
            file.write('\n')

def clean_tweet(tweet): 
    tweet = tweet.replace('\n',' ').replace('\r',' ')
    tweet = re.sub("\$[A-Z]*","<SYM>", tweet)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


def scrape(symbol, num_tweets=100, print_tweets=False):
    linesOfData = []
    tweets = []
    labels = []
    i = 0
    bulls = 0
    bears = 0
    page = 1
    totalTweets = 0
    
    ''' getting first page via IP rotation
        headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
        params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json"),);
        r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
    '''
    
    # getting first page without ip rotation.
    #r = requests.get('https://api.stocktwits.com/api/2/streams/symbol/' + symbol + ".json");

    jsonText = json.dumps(r.json())
    dictionary = json.loads(jsonText)
    maxi = dictionary['cursor']['max']
    length = len(dictionary['messages'])

    # number of tweets on the first page. 
    # if 30 then it works well
    #print(length)

    # master while loop can be configured 
    # either total labelled tweets or totalpages to scrape
    
    while totalTweets < num_tweets: 
            
        while i < length:
            
            isBull = 0; 
            isBear = 0; 
            isNull = 0; 
            
            message = dictionary['messages'][i]
            
            x = message['body'].replace("&#39;", "'").replace("&quot;","")
            
            x = clean_tweet(x)
            
            # dealing with the time the comment was posted. 
            strTime = message['created_at'].replace("T"," ").replace("Z","")
            dateTime = datetime.datetime.strptime(strTime, '%Y-%m-%d %H:%M:%S')
        
           
            if(str(dictionary['messages'][i]['entities']['sentiment']) == "None"):
                isNull = 0; 
        
            else:
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bullish":
                        bulls += 1
                        isBull = 1
                        totalTweets += 1
                                            
                if str(dictionary['messages'][i]['entities']['sentiment']['basic']) == "Bearish":
                        bears += 1        
                        isBear = 1 
                        totalTweets += 1
        
        
            if(isBull and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bullish ---------- ")
                    print(x+"\n\n")
                linesOfData.append(x+",Bullish")
                tweets.append(x)
                labels.append("Bullish")
                
            if(isBear and len(x) > 0):
                if print_tweets:
                    print(" ---------- "+str(totalTweets)+" | Page # "+str(page)+" | Tweet # "+str(i)+" | "+strTime+" | "+message['user']['username']+" | Bearish ----------")
                    print(x+"\n\n")
                linesOfData.append(x+",Bearish")
                tweets.append(x)
                labels.append("Bearish")
                
            i += 1
        
        suc = 0
        while suc < 1:
            try:
                
                # wait time for call in seconds. 
                time.sleep(0.5)
                
                ''' using the IP rotating proxy 
                headers = {"apikey": "bf892dc0-ba87-11ea-aa7d-4f92d4ba4d88"}
                params = (("url","https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi)),);
                r = requests.get('https://app.zenscrape.com/api/v1/get', headers=headers, params=params);
                '''
                
                # call without ip rotation. 
                #r = requests.get("https://api.stocktwits.com/api/2/streams/symbol/"+symbol+".json?max="+str(maxi));
                
                jsonText = json.dumps(r.json())
                
                dictionary = json.loads(jsonText)
                
                maxi = dictionary['cursor']['max']
                
                length = len(dictionary['messages'])
                
                i = 0
                
                suc = 1
                
            except JSONDecodeError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)

            except KeyError:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
                                        
            except:
                for x in sys.exc_info(): 
                    print("Unexpected error:", x)

                suc -= 3
                print("Retrying in "+str(suc*-1)+"s...")
                time.sleep(suc*-1)
            
        page += 1
    
    # TODO: Put data in database
    print(symbol + " scraped")
    
    #print("Printing to File")
    #printToFile(linesOfData)
    #print("Completed.")
    #return pd.DataFrame(list(zip(tweets, labels)), columns=["tweets", "labels"])

def mergeNewData(dfs=[]):
    if len(dfs) > 0:
        df1 = dfs[0]
        
        for df in dfs[0:]:
            df1 = df1.append(df, ignore_index=True)
        
        return df1

In [4]:
symbols = pd.read_csv("symbols.csv")
symbols = symbols["Symbol"]
random.choice(symbols)

'AAPL'

In [5]:
# threaded scraping loop
num_workers = 1

symbols_queue = symbols.tolist()


while symbols_queue:
    threads = []
    for x in range(num_workers):
        sym = symbols_queue.pop()
        t = threading.Thread(target=scrape, args=(sym,))
        threads.append(t)
    
    for t in threads:
        t.start()
    

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread     self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
Thread-5:
Exception in thread Thread-6Traceback (most recent call last):
:
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-7:
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
  File "/usr/lib/python

NameError: name 'r' is not defined

  File "/usr/lib/python3.8/threading.py", line 870, in run
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameError: name 'r' is not defined
NameError    : name 'r' is not defined
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
Exception in thread NameError: name 'r' is not defined
Thread-31:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
NameError: name 'r' is not defined
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread Thread-32:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-33:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threadin

self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameError: name 'r' is not defined
: Exception in thread Thread-54:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
name 'r' is not defined
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameErrorException in thread Thread-55:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run(): name 'r' is not defined
NameError: name 'r' is not defined
    NameError: name 'r' is not defined

  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread NameError: name 'r' is not defined
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Thread-56:
Traceback (most recent call last):
  File "/usr/lib/python3.8/

    Exception in thread Thread-80:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self.run()self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape

  File "/usr/lib/python3.8/threading.py", line 870, in run
NameError    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    NameError: name 'r' is not defined
NameErrorself.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
NameError: name 'r' is not defined
: name 'r' is not defined
: name 'r' is not defined
    Exception in thread Thread-81:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
NameError    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
: name 'r' is not defined
Exception in 

    : self._target(*self._args, **self._kwargs)    self._target(*self._args, **self._kwargs)
Exception in thread Thread-104:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
  File "/usr/lib/python3.8/threading.py", line 870, in run
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
name 'r' is not defined
      File "<ipython-input-3-7a421103127c>", line 44, in scrape
name 'r' is not defined
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
: self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Exception in thread Thread-105:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
name 'r' is not defined
        self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
Exception in thread Thread-106:
Traceback (most recent call la

...

: name 'r' is not defined
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
NameErrorNameError    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
: Exception in thread Thread-415:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
name 'r' is not defined
name 'r' is not defined
: name 'r' is not defined
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameErrorException in thread Thread-416:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
NameError: name 'r' is not defined
: name 'r' is not defined
    NameErrorself._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
Exception in thread     Thread-417:
Traceback (most recent call last):
: name 'r' is not defi

self._target(*self._args, **self._kwargs)    
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
        self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameError    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameError: name 'r' is not defined
Exception in thread Thread-439:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
: NameError: name 'r' is not defined
name 'r' is not defined
NameError: name 'r

    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
Thread-464:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape

  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    :     NameError    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
name 'r' is not defined
: name 'r' is not defined
NameErrorself.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    : name 'r' is not defined
Exception in thread Thread-465:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self._target(*self._args, **self._kwargs)E

    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self.run()Exception in thread : name 'r' is not defined

  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameErrorself.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    Thread-487:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameErrorNameError    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
Exception in thread Thread-488:
Traceback (most recent call last

self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameErrorNameError: name 'r' is not defined
self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
: name 'r' is not defined
NameError: name 'r' is not defined
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-7a421103127c>", line 44, in scrape
NameError: name 'r' is not defined
NameErrorNameError: name 'r' is not defined
: name 'r' is not defined
NameError: name 'r' is not defined
NameError: name 'r' is not defined
NameError: name 'r' is not defined
